<a href="https://colab.research.google.com/github/tera90223/random-walk-roomba-simulation/blob/main/MIT_Problem_Set_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive to access problem set
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Problem Set 3: Robot Simulation

## Problem 1: Implementing the RectangularRoom and Robot Abstract Classes

### Understanding Abstract Class in This Simulation

In object-oriented programming, an **abstract class** is a blueprint for other classes. It defines shared attributes and method signatures and leaves the actual implementation of certain methods to its subclasses.

Think of it like a contract:

> “Any class that inherits from me must implement these specific methods.”

You cannot create an instance of an abstract class directly. It exists to enforce structure, provide shared logic, and guide consistent subclass design.

#### Why Use Abstract Classes?
Abstract classes are powerful in simulations like this one because they let you:

* **Define shared behavior** (e.g., how to check if a position is in the room)
* **Enforce required methods** (e.g., update_position_and_clean)
* **Avoid duplicate code** while still allowing variety (e.g., different robot movement strategies)

For example:

* Every Robot has a position, direction, speed, and cleaning capacity.
* But not every robot moves the same way, so the movement logic is left abstract.

#### In This Project

* `RectangularRoom` provides the shared structure for room types but doesn't decide whether a room has furniture.
* `Robot` ensures all robots track position and clean tiles, but lets subclasses like `StandardRobot` or `FaultyRobot` define how they move and clean.

This design made the project modular and extensible which is ideal for comparing different movement and cleaning strategies without rewriting shared code.

In [ ]:
# -*- coding: utf-8 -*-
# Problem Set 3: Simulating robots
# Name: Chantera Lazard
# Time: 2 hours

import math
import random

import pylab

In [ ]:
# === Provided class Position
class Position(object):
    """
    A Position represents a location in a two-dimensional room, where
    coordinates are given by floats (x, y).
    """
    def __init__(self, x, y):
        """
        Initializes a position with coordinates (x, y).
        """
        self.x = x
        self.y = y

    def get_x(self):
        return self.x

    def get_y(self):
        return self.y

    def get_new_position(self, angle, speed):
        """
        Computes and returns the new Position after a single clock-tick has
        passed, with this object as the current position, and with the
        specified angle and speed.

        Does NOT test whether the returned position fits inside the room.

        angle: float representing angle in degrees, 0 <= angle < 360
        speed: positive float representing speed

        Returns: a Position object representing the new position.
        """
        old_x, old_y = self.get_x(), self.get_y()

        # Compute the change in position
        delta_y = speed * math.cos(math.radians(angle))
        delta_x = speed * math.sin(math.radians(angle))

        # Add that to the existing position
        new_x = old_x + delta_x
        new_y = old_y + delta_y

        return Position(new_x, new_y)

    def __str__(self):
        return "Position: " + str(math.floor(self.x)) + ", " + str(math.floor(self.y))


In [ ]:
# === Problem 1
class RectangularRoom(object):
    """
    A RectangularRoom represents a rectangular region containing clean or dirty
    tiles.

    A room has a width and a height and contains (width * height) tiles. Each tile
    has some fixed amount of dirt. The tile is considered clean only when the amount
    of dirt on this tile is 0.
    """
    def __init__(self, width, height, dirt_amount):
        """
        Initializes a rectangular room with the specified width, height, and
        dirt_amount on each tile.

        width: an integer > 0
        height: an integer > 0
        dirt_amount: an integer >= 0
        """
        # Width of the Rectangular Room
        self.width = width
        # Height of the Rectangular Room
        self.height = height
        # Initialize every tile with a certain amount of dirt
        for m in range(width):
            for n in range(height):
              self.tiles[(m,n)] = dirt_amount

    def clean_tile_at_position(self, pos, capacity):
        """
        Mark the tile under the position pos as cleaned by capacity amount of dirt.

        Assumes that pos represents a valid position inside this room.

        pos: a Position object
        capacity: the amount of dirt to be cleaned in a single time-step
                  can be negative which would mean adding dirt to the tile

        Note: The amount of dirt on each tile should be NON-NEGATIVE.
              If the capacity exceeds the amount of dirt on the tile, mark it as 0.
        """
        # Use position to get tile coordinates
        m = int(pos.get_x())
        n = int(pos.get_y())

        # If robot's cleaning capacity exceeds the amount of dirt on tile,
        # tile is cleaned
        dirt_amount = self.tile[(m,n)].dirt_amount
        if dirt_amount - capacity < 0:
            self.tiles[(m,n)]= 0
        else:
            #  Subtract robot's cleaning capacity from amount of dirt on tile
            dirt_amount -= capacity
            self.tiles[(m,n)] = dirt_amount

    def is_tile_cleaned(self, m, n):
        """
        Return True if the tile (m, n) has been cleaned.

        Assumes that (m, n) represents a valid tile inside the room.

        m: an integer
        n: an integer

        Returns: True if the tile (m, n) is cleaned, False otherwise

        Note: The tile is considered clean only when the amount of dirt on this
              tile is 0.
        """
        return self.tiles[(m,n)] == 0

    def get_num_cleaned_tiles(self):
        """
        Return the total number of clean tiles in the room.

        Returns: an integer
        """
        num_cleaned_tiles = 0
        # Check if each tile is cleaned (dirt_amount == 0)
        for tile in self.tiles:
            if self.tiles[tile] == 0:
                num_cleaned_tiles += 1
        return num_cleaned_tiles

    def is_position_in_room(self, pos):
        """
        Determines if pos is inside the room.

        pos: a Position object.
        Returns: True if pos is in the room, False otherwise.
        """
        x = pos.get_x()
        y = pos.get_y()
        if (int(x)) >= 0 and (int(x)) < (self.width-1) and \
         (int(y)) >= 0 and (int(y)) < (self.height-1):
            return True
        else:
            return False

    def get_dirt_amount(self, m, n):
        """
        Return the amount of dirt on the tile (m, n)

        Assumes that (m, n) represents a valid tile inside the room.

        m: an integer
        n: an integer

        Returns: an integer
        """
        return self.tiles[(m,n)]

    def get_num_tiles(self):
        """
        Returns: an integer; the total number of tiles in the room
        """
        # do not change -- implement in subclasses.

    def is_position_valid(self, pos):
        """
        pos: a Position object.

        returns: True if pos is in the room and (in the case of FurnishedRoom)
                 if position is unfurnished, False otherwise.
        """
        # do not change -- implement in subclasses

    def get_random_position(self):
        """
        Returns: a Position object; a random position inside the room
        """
        # do not change -- implement in subclasses

In [ ]:
class Robot(object):
    """
    Represents a robot cleaning a particular room.

    At all times, the robot has a particular position and direction in the room.
    The robot also has a fixed speed and a fixed cleaning capacity.

    Subclasses of Robot should provide movement strategies by implementing
    update_position_and_clean, which simulates a single time-step.
    """
    def __init__(self, room, speed, capacity):
        """
        Initializes a Robot with the given speed and given cleaning capacity in the
        specified room. The robot initially has a random direction and a random
        position in the room.

        room:  a RectangularRoom object.
        speed: a float (speed > 0)
        capacity: a positive interger; the amount of dirt cleaned by the robot
                  in a single time-step
        """
        self.room = room
        self.speed = speed
        self.capacity = capacity
        self.direction = random.random() * 360.0
        self.position = room.get_random_position()

    def get_robot_position(self):
        """
        Returns: a Position object giving the robot's position in the room.
        """
        return self.position

    def get_robot_direction(self):
        """
        Returns: a float d giving the direction of the robot as an angle in
        degrees, 0.0 <= d < 360.0.
        """
        return self.direction

    def set_robot_position(self, position):
        """
        Set the position of the robot to position.

        position: a Position object.
        """
        self.position = position

    def set_robot_direction(self, direction):
        """
        Set the direction of the robot to direction.

        direction: float representing an angle in degrees
        """
        self.direction = direction

    def update_position_and_clean(self):
        """
        Simulate the raise passage of a single time-step.

        Move the robot to a new random position (if the new position is invalid,
        rotate once to a random new direction, and stay stationary) and mark the tile it is on as having
        been cleaned by capacity amount.
        """
        # do not change -- implement in subclasses


## Problem 2: Implementing EmptyRoom and FurnishedRoom

### Notes on Room Inheritance & Design Decisions.

#### 1. `is_position_valid` as a Wrapper Method
The `is_position_valid` method serves as a wrapper around the more fundamental `is_position_in_room` method.

* `is_position_in_room` peforms a basic geometric check to ensure a position lies within the width and height boudnaries of the room.
* `is_position_valid` allows subclasses (like `FurnishedRoom`) to **customize position logic**, such as accounting furniture or obstacles.

This pattern demonstrates the power of method overriding to introduce class-specific behavior while preserving base functionality.



#### 2. Use of `RetangularRoom.__init__` vs `super()`

In the `FurnishedRoom` class, the parent constructor is called using:

```
RectangularRoom.__init__(self, width, height, dirt_amount)
```

While this works, a more modern and flexible approach in Python would be:

```
super().__init__(width, height, dirt_amount)
```

Using `super()`:
* Adheres to best practices
* Supports multiple inheritence scenarios
* Avoids hardcoding the parent class name

In Structured course settings, the explicit call to `RectangularRoom` may be preferred for clarity, but `super()` is generally recommended in professional Python code.

In [ ]:
# === Problem 2
class EmptyRoom(RectangularRoom):
    """
    An EmptyRoom represents a RectangularRoom with no furniture.
    """
    def get_num_tiles(self):
        """
        Returns: an integer; the total number of tiles in the room
        """
        return self.width * self.height

    def is_position_valid(self, pos):
        """
        pos: a Position object.

        Returns: True if pos is in the room, False otherwise.
        """
        return self.is_position_in_room(pos)

    def get_random_position(self):
        """
        Returns: a Position object; a valid random position (inside the room).
        """
        x = random.random() * (self.width-1)
        y = random.random() * (self.height-1)
        return Position(x, y)

In [ ]:
class FurnishedRoom(RectangularRoom):
    """
    A FurnishedRoom represents a RectangularRoom with a rectangular piece of
    furniture. The robot should not be able to land on these furniture tiles.
    """
    def __init__(self, width, height, dirt_amount):
        """
        Initializes a FurnishedRoom, a subclass of RectangularRoom. FurnishedRoom
        also has a list of tiles which are furnished (furniture_tiles).
        """
        # This __init__ method is implemented for you -- do not change.
        # Call the __init__ method for the parent class
        RectangularRoom.__init__(self, width, height, dirt_amount)
        # Adds the data structure to contain the list of furnished tiles
        self.furniture_tiles = []

    def add_furniture_to_room(self):
        """
        Add a rectangular piece of furniture to the room. Furnished tiles are stored
        as (x, y) tuples in the list furniture_tiles

        Furniture location and size is randomly selected. Width and height are selected
        so that the piece of furniture fits within the room and does not occupy the
        entire room. Position is selected by randomly selecting the location of the
        bottom left corner of the piece of furniture so that the entire piece of
        furniture lies in the room.
        """
        # This addFurnitureToRoom method is implemented for you. Do not change it.
        furniture_width = random.randint(1, self.width - 1)
        furniture_height = random.randint(1, self.height - 1)

        # Randomly choose bottom left corner of the furniture item.
        f_bottom_left_x = random.randint(0, self.width - furniture_width)
        f_bottom_left_y = random.randint(0, self.height - furniture_height)

        # Fill list with tuples of furniture tiles.
        for i in range(f_bottom_left_x, f_bottom_left_x + furniture_width):
            for j in range(f_bottom_left_y, f_bottom_left_y + furniture_height):
                self.furniture_tiles.append((i,j))

    def is_tile_furnished(self, m, n):
        """
        Return True if tile (m, n) is furnished.
        """
        if (m, n) in self.furniture_tiles:
          return True
        else:
          return False

    def is_position_furnished(self, pos):
        """
        pos: a Position object.

        Returns True if pos is furnished and False otherwise
        """
        m = int(pos.get_x())
        n = int(pos.get_y())
        if self.is_tile_furnished(m,n):
          return True
        else:
          return False

    def is_position_valid(self, pos):
        """
        pos: a Position object.

        returns: True if pos is in the room and is unfurnished, False otherwise.
        """
        if self.is_position_in_room(pos) and not self.is_position_furnished(pos):
          return True
        else:
          return False

    def get_num_tiles(self):
        """
        Returns: an integer; the total number of tiles in the room that can be accessed.
        """
        return self.width * self.height - len(self.furniture_tiles)

    def get_random_position(self):
        """
        Returns: a Position object; a valid random position (inside the room and not in a furnished area).
        """
        # This starts an infinite loop that will return position when criteria
        # is met
        while True:
          x = random.random() * (self.width-1)
          y = random.random() * (self.height-1)
          pos = Position(x, y)
          if self.is_position_valid(pos):
            return pos
